<a href="https://colab.research.google.com/github/adarshlearnngrow/StepUp-AI/blob/Base-commit/AI_Hackthon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import openai
import json
import time
from openai import OpenAI
from google.colab import userdata
import pandas as pd


In [ ]:
pd.set_option('display.max_colwidth', None)


In [ ]:
open_ai_client = OpenAI(api_key=userdata.get("OPEN_AI_KEY"))


In [ ]:
def generate_resume_gap_example():
    prompt = """
Generate synthetic data for training a resume-to-job gap detection model.

Respond in this JSON format:
{
  "input": {
    "resume": "<full resume with sections: education, work experience, personal projects, technical skills, soft skills>",
    "job_description": "<realistic job description>"
  },
  "output": {
    "missing_skills": ["<skill1>", "<skill2>", "..."]
  }
}

Ensure:
- Resume is realistic and includes relevant but incomplete skills.
- Job description targets a specific role (e.g. 'Machine Learning Engineer').
- Output contains only truly missing skills not in the resume.
Only return valid JSON. No markdown, no explanation.
"""
    response = open_ai_client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )
    return response.choices[0].message.content.strip()

# Generate dataset
output_file = "resume_gap_dataset.jsonl"

with open(output_file, "w") as f:
    for i in range(5):  # Generate 100 examples
        try:
            result = generate_resume_gap_example()
            parsed = json.loads(result)  # Ensure it's valid JSON
            f.write(json.dumps(parsed) + "\n")
            print(f"✅ Generated {i+1}")
            time.sleep(1)  # avoid rate limit
        except json.JSONDecodeError:
            print(f"⚠️ JSON parsing failed at {i+1}, output:\n{result}\n")
        except Exception as e:
            print(f"❌ API or file error at {i+1}: {e}")

✅ Generated 1
✅ Generated 2
✅ Generated 3
✅ Generated 4
✅ Generated 5
